# Instalação de dependências

In [2]:
# !pip install pandas
# !pip install python-dotenv
# !pip install openai
# !pip install replicate

# Importação das bibliotecas

In [3]:
# Importação das bibliotecas
import replicate
import os
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

True

# Configurações do Replicate

In [4]:
# Autenticação no Replicate
replicate_api_key = os.getenv("REPLICATE_API_KEY") # obtêm token no .env

# Salvar token cliente em variável
client = replicate.Client(api_token=replicate_api_key)

In [5]:
# Nome do modelo de IA utilizado
model_name = "meta/meta-llama-3-8b-instruct"

# Obter respostas do LLM

In [24]:
# Função que recebe o prompt e retorna a resposta do modelo
def get_chatbot_response(client, model_name, messages, temperature=0):
    # Lista das mensagens de entrada do usuário
    input_messages = []
    for message in messages:
        input_messages.append(message)
    # 
    model_response = {
        "prompt": "Finja ser o chatbot de um restaurante.",  # prompt de entrada
        # número máximo de tokens (palavras ou sub-palavras) a serem gerados
        "max_new_tokens": 2_000,
        # temperatura de amostragem: nenhuma aleatoriedade (queremos resultados concretos)
        "temperature": 0.0,
        # template de prompt:
        "prompt_template": "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n",
        "top_p": 0.8,  # amostragem de núcleo: 80% dos tokens mais prováveis
    }
    # Adicionar cada token à string de saída
    response = ""
    for event in client.stream(model_name,input=model_response):
        response += str(event)  # Adiciona cada token à string de saída
    # Retorna a resposta do modelo
    return response


In [25]:
# Teste da função
msg_usuario="Qual é o prato do dia?"
resposta = get_chatbot_response(client, model_name, msg_usuario)

print(f"\nUsuário: {msg_usuario}\nChatBot: {resposta}")


Usuário: Qual é o prato do dia?
ChatBot: Olá! Bem-vindo ao Restaurante "Sabor e Alegria"! Estou aqui para ajudá-lo com qualquer dúvida ou solicitação que tenha. O que posso fazer por você hoje?

Você está procurando por informações sobre o nosso cardápio, reservar uma mesa, ou talvez fazer uma encomenda para entrega ou retirada?
